In [6]:
#!pip install textblob
#!pip install langdetect

  Stored in directory: C:\Users\peter\AppData\Local\pip\Cache\wheels\ec\0c\a9\1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [1]:
#imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
import nltk
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import multiprocessing
import gensim
from gensim.utils import simple_preprocess

In [2]:
songdata = pd.read_csv('E:\\Roy Master File\\Winter2020\\Data Mining\\final\\Final Notebooks\\songdata.csv')

In [3]:
songdata.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df = pd.read_pickle("E:\\Roy Master File\\Winter2020\\Data Mining\\final\\Final Notebooks\\genres.pkl")
df = pd.merge(songdata, df, on=['artist','artist'])
df = df[df.iloc[:,5:].any(axis = 1)]

In [7]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","[europop, swedish pop]",0,0,1,0,0,0,0
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","[europop, swedish pop]",0,0,1,0,0,0,0
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,"[europop, swedish pop]",0,0,1,0,0,0,0
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51489 entries, 0 to 57595
Data columns (total 12 columns):
artist               51489 non-null object
song                 51489 non-null object
link                 51489 non-null object
text                 51489 non-null object
genres               51489 non-null object
rock                 51489 non-null int32
singer-songwriter    51489 non-null int32
pop                  51489 non-null int32
metal                51489 non-null int32
folk                 51489 non-null int32
country              51489 non-null int32
hip hop / rap        51489 non-null int32
dtypes: int32(7), object(5)
memory usage: 3.7+ MB


We have a multilabel classification problem according to https://scikit-learn.org/stable/modules/multiclass.html

Multilabel- 

"A multiclass multioutput target where each output is binary. This may be represented as a 2d (dense) array or sparse matrix of integers, such that each column is a separate binary target, where positive labels are indicated with 1 and negative labels are usually -1 or 0. Sparse multilabel targets are not supported everywhere that dense multilabel targets are supported."

"Valid representation of multilabel y is either dense (or sparse) binary matrix of shape (n_samples, n_classes). Each column represents a class. The 1’s in each row denote the positive classes a sample has been labelled with."

In [9]:
#here is y in array form if you want it.
#this would have happened in a train_test_split anyway
y = np.array(df.iloc[:,5:])

Some text is in round brackets while some is in square brackets. I wanted to examine what that text looked like.

In [10]:
#examining text in round brackets
round_brackets = sum(list(df['text'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
#Number of round brackets:
print((len(round_brackets)))


54962


In [11]:
#Viewing some text in round brackets
#These just look like normal lyrics
random.seed(0)
random.choices(round_brackets, k=15)

['I knew',
 'tell it',
 'gunshots',
 "feel like I'm back",
 'Judy',
 'we were in the park',
 "You don't own",
 'repeat to fade',
 "it's alright",
 'Originally recorded by Diamond Head',
 'or should I have my green eyes?',
 'Dark night',
 'vocals',
 'I got 1,000 hugs and kisses 4 U when U come back home, baby',
 "But that's alright"]

In [12]:
#examining text in square brackets
square_brackets = sum(list(df['text'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
#how many instances of square brackets?
print((len(square_brackets)))



26001


In [13]:
#re.sub(pattern, repl, string, count=0, flags=0)
# remove round brackets but not text within
df['text'] = df['text'].map(lambda s: re.sub(r'\(|\)', '', s))

# remove square brackest and text within
df['text'] = df['text'].map(lambda s: re.sub(r'\[(.*?)\] ', '', s))

In [14]:
# remove line breaks
df['text'] = df['text'].map(lambda s: re.sub(r' \n|\n', '', s))

### Removing Non-English Songs

Note: This takes a while to run. 

In [15]:
#This find the probability that the word is english
def get_eng_prob(text):
    detections = detect_langs(text)
    for detection in detections:
        if detection.lang == 'en':
            return detection.prob
    return 0

#finding the probability that the text is english
df['en_prob'] = df['text'].map(get_eng_prob)

print('Number of songs in english: {}'.format(sum(df['en_prob'] >= 0.5)))
print('Number of songs that are not english: {}'.format(sum(df['en_prob'] < 0.5)))

Number of songs in english: 51129
Number of songs that are not english: 360


In [16]:
#only selecting songs that have a probability of 0.5 or higher of being in english
df = df.loc[df['en_prob'] >= 0.5]

In [17]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,en_prob
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face And it...","[europop, swedish pop]",0,0,1,0,0,0,0,0.999996
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently l...","[europop, swedish pop]",0,0,1,0,0,0,0,0.999996
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go Why I had to p...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999998
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999996
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999997


In [18]:
#Repeating above code to make sure that the non-english songs were dropped
print('Number of songs in english: {}'.format(sum(df['en_prob'] >= 0.5)))
print('Number of songs that are not english: {}'.format(sum(df['en_prob'] < 0.5)))

Number of songs in english: 51129
Number of songs that are not english: 0


In [19]:
df = df.drop(['en_prob'], axis=1)

### Tokenization

Using nltk.tokenize to seperate the text into a list of words. All punctuation is removed. 

In [21]:
#w+ for whitespace
#tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

#creating a new column called tokens
#df['tokens'] = df['text'].map(tokenizer.tokenize)

We originally cleaned the data using the tokenization text above. After later running the word2vec, we didn't like that the words were not lowercased, and thought they could be cleaned up better. We are proceeding with the code below instead. According to documentation, this uses tokenize() internally.

https://radimrehurek.com/gensim/utils.html

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Roy's\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [24]:
df['text_preprocessed'] = df['text'].map(preprocess)

In [25]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face And it...","[europop, swedish pop]",0,0,1,0,0,0,0,look at her face it wonder face and it mean so...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently l...","[europop, swedish pop]",0,0,1,0,0,0,0,take it easi with me pleas touch me gentli lik...
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go Why I had to p...,"[europop, swedish pop]",0,0,1,0,0,0,0,ll never know whi have to go whi have to put u...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...


In [26]:
#Checking what the tokens column look like before removing stopwords
df['text_preprocessed'].head()

0    look at her face it wonder face and it mean so...
1    take it easi with me pleas touch me gentli lik...
2    ll never know whi have to go whi have to put u...
3    make somebodi happi be question of give and ta...
4    make somebodi happi be question of give and ta...
Name: text_preprocessed, dtype: object

In [128]:
#safety
df2 = df

In [27]:
#w+ for whitespace
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

#creating a new column called tokens
df['tokens'] = df['text_preprocessed'].map(tokenizer.tokenize)

In [28]:
df

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face And it...","[europop, swedish pop]",0,0,1,0,0,0,0,look at her face it wonder face and it mean so...,"[look, at, her, face, it, wonder, face, and, i..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently l...","[europop, swedish pop]",0,0,1,0,0,0,0,take it easi with me pleas touch me gentli lik...,"[take, it, easi, with, me, pleas, touch, me, g..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go Why I had to p...,"[europop, swedish pop]",0,0,1,0,0,0,0,ll never know whi have to go whi have to put u...,"[ll, never, know, whi, have, to, go, whi, have..."
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, be, question, of, give..."
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, be, question, of, give..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57588,Van Der Graaf Generator,Man-Erg,/v/van+der+graaf+generator/man+erg_20260114.html,The killer lives inside me: I can feel him mov...,"[art rock, canterbury scene, experimental, jaz...",1,0,0,0,0,0,0,the killer live insid me can feel him move som...,"[the, killer, live, insid, me, can, feel, him,..."
57589,Van Der Graaf Generator,Mirror Images,/v/van+der+graaf+generator/mirror+images_20263...,If I'm the mirror and you're the image Then wh...,"[art rock, canterbury scene, experimental, jaz...",1,0,0,0,0,0,0,if the mirror and you re the imag then what th...,"[if, the, mirror, and, you, re, the, imag, the..."
57590,Van Der Graaf Generator,Sleepwalkers,/v/van+der+graaf+generator/sleepwalkers_202430...,"At night, this mindless army, ranks unbroken b...","[art rock, canterbury scene, experimental, jaz...",1,0,0,0,0,0,0,at night thi mindless armi rank unbroken by di...,"[at, night, thi, mindless, armi, rank, unbroke..."
57594,Zazie,Duo,/z/zazie/duo_20720322.html,Oui Je sens le vent Je sens la pluie Ressens l...,"[chanson, french pop, french rock, nouvelle ch...",1,0,1,0,0,0,0,oui je sen le vent je sen la pluie ressen la p...,"[oui, je, sen, le, vent, je, sen, la, pluie, r..."


### Removing stop words
Reference for more info: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [29]:
#setting stop words

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Roy's\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
#removing stop words
df['tokens'] = df['tokens'].apply(lambda x: [item for item in x if item not in stop_words])


In [31]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face And it...","[europop, swedish pop]",0,0,1,0,0,0,0,look at her face it wonder face and it mean so...,"[look, face, wonder, face, mean, someth, speci..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently l...","[europop, swedish pop]",0,0,1,0,0,0,0,take it easi with me pleas touch me gentli lik...,"[take, easi, pleas, touch, gentli, like, summe..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go Why I had to p...,"[europop, swedish pop]",0,0,1,0,0,0,0,ll never know whi have to go whi have to put u...,"[never, know, whi, go, whi, put, lousi, rotten..."
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, question, give, take, ..."
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, question, give, take, ..."


### Stemming the data - stemming done above in pre-processing

In [36]:
# import these modules 
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
 
# create dictionary to map tokens their lem
token_to_lem = {}
# initialise word count
token_count = 0


# iterate through all songs
for lst in df['tokens']:
    # iterate through all tokens of song
    for token in lst:
        token_count += 1
        # check if token is in dictionary
        if token not in token_to_lem:
            # add token to dictionary
            token_to_lem[token] = lemmatizer.lemmatize(token)
    
    
#creating column stems            
df['stems'] = df['tokens'].map(lambda lst: [token_to_lem[token] for token in lst])

We give the option to either lemmatization or stemming. We ran both and compared our outputs.

In [37]:
#intialize stemmer
stemmer = PorterStemmer()

# create dictionary to map tokens their stem
token_to_stem = {}
# initialise word count
token_count = 0


# iterate through all songs
for lst in df['tokens']:
    # iterate through all tokens of song
    for token in lst:
        token_count += 1
        # check if token is in dictionary
        if token not in token_to_stem:
            # add token to dictionary
            token_to_stem[token] = stemmer.stem(token)

#creating column stems            
df['stems'] = df['tokens'].map(lambda lst: [token_to_stem[token] for token in lst])


In [38]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stems
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face And it...","[europop, swedish pop]",0,0,1,0,0,0,0,look at her face it wonder face and it mean so...,"[look, face, wonder, face, mean, someth, speci...","[look, face, wonder, face, mean, someth, speci..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently l...","[europop, swedish pop]",0,0,1,0,0,0,0,take it easi with me pleas touch me gentli lik...,"[take, easi, pleas, touch, gentli, like, summe...","[take, easi, plea, touch, gentli, like, summer..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go Why I had to p...,"[europop, swedish pop]",0,0,1,0,0,0,0,ll never know whi have to go whi have to put u...,"[never, know, whi, go, whi, put, lousi, rotten...","[never, know, whi, go, whi, put, lousi, rotten..."
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, question, give, take, ...","[make, somebodi, happi, question, give, take, ..."
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, question, give, take, ...","[make, somebodi, happi, question, give, take, ..."


In [39]:
#checking out the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51129 entries, 0 to 57595
Data columns (total 15 columns):
artist               51129 non-null object
song                 51129 non-null object
link                 51129 non-null object
text                 51129 non-null object
genres               51129 non-null object
rock                 51129 non-null int32
singer-songwriter    51129 non-null int32
pop                  51129 non-null int32
metal                51129 non-null int32
folk                 51129 non-null int32
country              51129 non-null int32
hip hop / rap        51129 non-null int32
text_preprocessed    51129 non-null object
tokens               51129 non-null object
stems                51129 non-null object
dtypes: int32(7), object(8)
memory usage: 4.9+ MB


In [40]:
#number of tokens
print(token_count)

5623928


In [41]:
#number of unique tokens
len(token_to_stem.keys())

51151

In [42]:
#number of unique stems
(len(set(token_to_stem.values())))

50676

In [43]:
#turning the stems back into untokenized structrure so that we can run count vector on it
df['stem_str'] = df['tokens'].map(lambda lst: ' '.join(lst))

In [44]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stems,stem_str
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face And it...","[europop, swedish pop]",0,0,1,0,0,0,0,look at her face it wonder face and it mean so...,"[look, face, wonder, face, mean, someth, speci...","[look, face, wonder, face, mean, someth, speci...",look face wonder face mean someth special look...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently l...","[europop, swedish pop]",0,0,1,0,0,0,0,take it easi with me pleas touch me gentli lik...,"[take, easi, pleas, touch, gentli, like, summe...","[take, easi, plea, touch, gentli, like, summer...",take easi pleas touch gentli like summer even ...
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go Why I had to p...,"[europop, swedish pop]",0,0,1,0,0,0,0,ll never know whi have to go whi have to put u...,"[never, know, whi, go, whi, put, lousi, rotten...","[never, know, whi, go, whi, put, lousi, rotten...",never know whi go whi put lousi rotten show bo...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, question, give, take, ...","[make, somebodi, happi, question, give, take, ...",make somebodi happi question give take learn s...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, question, give, take, ...","[make, somebodi, happi, question, give, take, ...",make somebodi happi question give take learn s...


### Vectorization

In [64]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from langdetect import detect
# from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import svm
import plotly.graph_objects as go
# from better_profanity import profanity
import string

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [52]:
df['tokens2']=df.tokens.apply(lambda x:str(x).strip('[]'))
df.head()                  

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stems,stem_str,tokens2
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face And it...","[europop, swedish pop]",0,0,1,0,0,0,0,look at her face it wonder face and it mean so...,"[look, face, wonder, face, mean, someth, speci...","[look, face, wonder, face, mean, someth, speci...",look face wonder face mean someth special look...,"'look', 'face', 'wonder', 'face', 'mean', 'som..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently l...","[europop, swedish pop]",0,0,1,0,0,0,0,take it easi with me pleas touch me gentli lik...,"[take, easi, pleas, touch, gentli, like, summe...","[take, easi, plea, touch, gentli, like, summer...",take easi pleas touch gentli like summer even ...,"'take', 'easi', 'pleas', 'touch', 'gentli', 'l..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go Why I had to p...,"[europop, swedish pop]",0,0,1,0,0,0,0,ll never know whi have to go whi have to put u...,"[never, know, whi, go, whi, put, lousi, rotten...","[never, know, whi, go, whi, put, lousi, rotten...",never know whi go whi put lousi rotten show bo...,"'never', 'know', 'whi', 'go', 'whi', 'put', 'l..."
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, question, give, take, ...","[make, somebodi, happi, question, give, take, ...",make somebodi happi question give take learn s...,"'make', 'somebodi', 'happi', 'question', 'give..."
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,make somebodi happi be question of give and ta...,"[make, somebodi, happi, question, give, take, ...","[make, somebodi, happi, question, give, take, ...",make somebodi happi question give take learn s...,"'make', 'somebodi', 'happi', 'question', 'give..."


#### CountVectorizer

In [53]:
cv = CountVectorizer()

stem_count_vector = cv.fit_transform(df['tokens2'])

In [102]:
X_train, X_test, y_train, y_test = train_test_split(df.tokens2, df['pop'], random_state = 0)

In [103]:
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [104]:
nb = MultinomialNB()
nb.fit(X_train_cv, y_train)
y_pred = nb.predict(X_test_cv)

In [105]:
print('Multinomial Naive Bayes Accuracy:', accuracy_score(y_test, y_pred))

Multinomial Naive Bayes Accuracy: 0.6352968786669796


In [106]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67      7097
           1       0.59      0.59      0.59      5686

    accuracy                           0.64     12783
   macro avg       0.63      0.63      0.63     12783
weighted avg       0.64      0.64      0.64     12783



In [67]:
n = [17, 19, 20]

for i in n:
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train_cv, y_train)
    y_pred_knn = knn.predict(X_test_cv)
    print('K-Nearest Neighbors Accuracy for k=', i, ':', accuracy_score(y_test, y_pred_knn))

K-Nearest Neighbors Accuracy for k= 17 : 0.7581944770398185
K-Nearest Neighbors Accuracy for k= 19 : 0.7574121880622702
K-Nearest Neighbors Accuracy for k= 20 : 0.7572557302667605


In [70]:
X_train.shape

(38346,)

In [71]:
X_test.shape

(12783,)

#### Tf-Idf

In [80]:
tfidf = TfidfVectorizer()
X_train_cv = tfidf.fit_transform(X_train)
X_test_cv = tfidf.transform(X_test)

In [81]:
nb = MultinomialNB()
nb.fit(X_train_cv, y_train)
y_pred = nb.predict(X_test_cv)

In [82]:
print('Multinomial Naive Bayes Accuracy:', accuracy_score(y_test, y_pred))

Multinomial Naive Bayes Accuracy: 0.8506610341860283


### Text Analysis

In [152]:
#Documentation: 
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html


# initialise count vectorizer
#Convert a collection of text documents to a matrix of token counts
#This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.
#cv = CountVectorizer()

# generate word counts
#stem_count_vector = cv.fit_transform(df['stem_str'])


In [153]:
#stem_count_vector

<57175x50762 sparse matrix of type '<class 'numpy.int64'>'
	with 3334732 stored elements in Compressed Sparse Row format>

In [166]:
EMB_DIM=300
wv_model=Word2Vec(df["tokens"],
             size=EMB_DIM,
             window=5,
             min_count=5, 
             negative=15, 
             iter=10,
             workers=multiprocessing.cpu_count(), 
             #defines the algorithm to use as the model. sg = skip-gram
             #default is CBOW (Continuous bag of words)
             sg = 1   )

In [188]:
# load the model from disk
import pickle
  
# Save the trained model as a pickle string. 
pickle.dump(wv_model, open( "songs_wv_model", "wb" ))

In [167]:
word_vectors=wv_model.wv

### Exploring different types of word similarity

In [182]:
#Exploring similar words
#calling object similar_by_word
word_vectors.similar_by_word('masterplan')

[('lorain', 0.6302295923233032),
 ('couldv', 0.6193580627441406),
 ('distantli', 0.6171380877494812),
 ('overreact', 0.6133875250816345),
 ('unconvent', 0.6129610538482666),
 ('unmerit', 0.6109627485275269),
 ('questionin', 0.6030411720275879),
 ('connot', 0.6023432016372681),
 ('loophol', 0.5982478260993958),
 ('emigr', 0.5925590991973877)]

In [ ]:
#vectors.most_similar(positive=['insertwordhere'],negative=['insertwordhere'])

In [183]:
#showing vector of individual words
wv_model["love"]

/Users/breabeals/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



array([-2.24736318e-01,  1.97534621e-01, -4.29814868e-02,  5.71665466e-02,
        5.65432794e-02,  1.85361847e-01, -6.85141310e-02,  8.89955685e-02,
        1.21765053e-02,  2.06344709e-01, -1.14845835e-01, -1.25641763e-01,
       -4.58884798e-03, -1.42729923e-03,  1.43025428e-01, -3.46773177e-01,
        1.35805786e-01, -2.56192654e-01, -1.26809895e-01, -1.30737841e-01,
        9.08707082e-02,  1.35844260e-01, -5.43559194e-02,  4.73242477e-02,
        2.51561612e-01, -1.24103695e-01, -1.26703739e-01, -1.88817494e-02,
        3.10138404e-01, -3.21486928e-02, -2.03293823e-02,  1.41334394e-02,
       -1.42418832e-01, -6.57715797e-02, -1.12045988e-01,  1.60259008e-01,
        1.85000822e-02,  1.49633542e-01, -8.46545473e-02, -9.50547680e-03,
        5.91595583e-02, -1.16509996e-01,  4.31440026e-03,  3.29046361e-02,
       -1.00284070e-01, -9.64479744e-02,  2.31143460e-01, -1.24456011e-01,
        7.63623789e-02, -2.73124706e-02,  1.17235025e-02,  1.30105345e-03,
        1.76290572e-01,  

In [187]:
wv_model.similarity('love', 'hate')

/Users/breabeals/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).



0.31420258

In [190]:
df.to_csv('Processed_Songs_Data.csv')